In [1]:
import sys
sys.path.append("utils")
import dynamic_backtesting as db
import compare_strategies as cs
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["grey", "green", "black", "deepskyblue", "orange", "lime", "purple", "Gold", "brown", "blue", "pink"]) 

In [2]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
stocks = tickers[:7]
commodities = tickers[7:9]
bonds = tickers[9:]

**Laver funktion til at hente vægte og udregne performance for forskellige tickers og gamma**

In [3]:
def backtesting(tickers,Gamma=1.526988705495546e-06):
    v_t_ARCH_TC, out_of_sample_ARCH_TC, _,Omega_t_ARCH_TC = db.garch_with_trading_cost(tickers=tickers,
    number_of_out_of_sample_days=4*250, model_type="sGARCH10",tuning_gamma_D=Gamma)
    v_t_GARCH_TC, out_of_sample_GARCH_TC, _,Omega_t_GARCH_TC = db.garch_with_trading_cost(tickers=tickers,
    number_of_out_of_sample_days=4*250, model_type="sGARCH11",tuning_gamma_D=Gamma)
    v_t_GJR_TC, out_of_sample_GJR_TC, _,Omega_t_GJR_TC = db.garch_with_trading_cost(tickers=tickers,
    number_of_out_of_sample_days=4*250, model_type="gjrGARCH11",tuning_gamma_D=Gamma)
    res_ARCH_TC, perf_table_ARCH_TC = cs.performance_table(v_t_ARCH_TC, out_of_sample_ARCH_TC,Omega_t_ARCH_TC,portfolio_value=1e9)
    res_GARCH_TC, perf_table_GARCH_TC = cs.performance_table(v_t_GARCH_TC, out_of_sample_GARCH_TC,Omega_t_GARCH_TC,portfolio_value=1e9)
    res_GJR_TC, perf_table_GJR_TC = cs.performance_table(v_t_GJR_TC, out_of_sample_GJR_TC,Omega_t_GJR_TC,portfolio_value=1e9)
    perf_table_TC=perf_table_ARCH_TC.iloc[0:1,]
    perf_table_TC=perf_table_TC.rename(index={'GARCH':'ARCH(1)'})
    perf_table_TC=perf_table_TC.append(perf_table_GARCH_TC.iloc[0,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH':'GARCH(1,1)'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[0,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH':'GJR-GARCH(1,1)'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[2,])
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[1,])
    perf_table_TC=perf_table_TC.rename(index={'BnH':'Buy and hold','Equal_weight':'Equal weight'})
    #Trading cost
    perf_table_TC=perf_table_TC.append(perf_table_ARCH_TC.iloc[3,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH TC':'ARCH(1) TC'})
    perf_table_TC=perf_table_TC.append(perf_table_GARCH_TC.iloc[3,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH TC':'GARCH(1,1) TC'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[3,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH TC':'GJR-GARCH(1,1) TC'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[4,])
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[5,])
    perf_table_TC=perf_table_TC.rename(index={'BnH TC':'Buy and hold TC','Equal_weight TC':'Equal weight TC'})

    #gross and net return
    res_tc=res_ARCH_TC.drop(columns=['GARCH TC','Equal_weight', 'BnH','Equal_weight TC','BnH TC']).rename(columns={'GARCH':'ARCH(1)'})
    res_tc["GARCH(1,1)"]=res_GARCH_TC["GARCH"]
    res_tc["GJR-GARCH(1,1)"]=res_GJR_TC["GARCH"]
    res_tc["Equal weight"]=res_GJR_TC["Equal_weight"]
    res_tc["Buy and hold"]=res_GJR_TC["BnH"]

    res_tc_TC=res_ARCH_TC.drop(columns=['GARCH','Equal_weight', 'BnH','Equal_weight TC','BnH TC']).rename(columns={'GARCH TC':'ARCH(1)'})
    res_tc_TC["GARCH(1,1)"]=res_GARCH_TC["GARCH TC"]
    res_tc_TC["GJR-GARCH(1,1)"]=res_GJR_TC["GARCH TC"]
    res_tc_TC["Equal weight"]=res_GJR_TC["Equal_weight TC"]
    res_tc_TC["Buy and hold"]=res_GJR_TC["BnH TC"]
    return v_t_ARCH_TC,v_t_GARCH_TC, v_t_GJR_TC, perf_table_TC,res_tc,res_tc_TC


**All assets, calibrated gamma**

In [ ]:
v_t_ARCH_all_low,v_t_GARCH_all_low, v_t_GJR_all_low, perf_table_all_low,res_all_low,res_all_low_TC = backtesting(tickers)

**Figurer**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(res_all_low,label=res_all_low.columns,linewidth=1)
ax.margins(0,0.05)
ax.legend(res_all_low.columns)
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Gross Return',loc='top', rotation = 0, labelpad = -88) 
#fig.savefig("Performance_TC_before_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(res_all_low_TC,label=res_all_low_TC.columns,linewidth=1)
ax.margins(0,0.05)
ax.legend(res_all_low_TC.columns)
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Net Return',loc='top', rotation = 0, labelpad = -76) 
#fig.savefig("Performance_TC_after_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_ARCH_all_low,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("ARCH_weight_TC_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_GARCH_all_low,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("GARCH_weight_TC_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_GJR_all_low,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("GJRGARCH_weight_TC_L.png", bbox_inches = 'tight', pad_inches = 0.1)

**All assets, lowest std**

In [ ]:
v_t_ARCH_all_std,v_t_GARCH_all_std, v_t_GJR_all_std, perf_table_all_std,res_all_std,res_all_std_TC = backtesting(tickers,0.000023)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(res_all_std,label=res_all_std.columns,linewidth=1)
ax.margins(0,0.05)
ax.legend(res_all_std.columns)
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Gross Return',loc='top', rotation = 0, labelpad = -88) 
#fig.savefig("Performance_TC_before_LSTD.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(res_all_std_TC,label=res_all_std_TC.columns,linewidth=1)
ax.margins(0,0.05)
ax.legend(res_all_std_TC.columns)
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Net Return',loc='top', rotation = 0, labelpad = -76) 
#fig.savefig("Performance_TC_after_LSTD.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_ARCH_all_std,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("ARCH_weight_TC_LSTD.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_GARCH_all_std,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("GARCH_weight_TC_LSTD.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_GJR_all_std,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("GJRGARCH_weight_TC_LSTD.png", bbox_inches = 'tight', pad_inches = 0.1)

**All assets, combination between return and risk**

In [4]:
v_t_ARCH_all_best,v_t_GARCH_all_best, v_t_GJR_all_best, perf_table_all_best,res_all_best,res_all_best_TC = backtesting(tickers,1e-5)

['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1e-05
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1e-05
['EEM', 'IVV', 'IEV', 'IXN', 'IYR', 'IXG', 'EXI', 'GC=F', 'BZ=F', 'HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1e-05


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(res_all_best,label=res_all_best.columns,linewidth=1)
ax.margins(0,0.05)
ax.legend(res_all_best.columns)
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Gross Return',loc='top', rotation = 0, labelpad = -88) 
#fig.savefig("Performance_TC_before_best.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(res_all_best_TC,label=res_all_best_TC.columns,linewidth=1)
ax.margins(0,0.05)
ax.legend(res_all_best_TC.columns)
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Net Return',loc='top', rotation = 0, labelpad = -76) 
#fig.savefig("Performance_TC_after_best.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_ARCH_all_best,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("ARCH_weight_TC_best.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_GARCH_all_best,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("GARCH_weight_TC_best.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=190)
ax.plot(v_t_GJR_all_best,linewidth=0.8)
ax.margins(0,0.05)
ax.legend(assets.values(),ncol=3,loc="upper left")
ax.xaxis.set_major_locator(plt.MaxNLocator(6))
ax.set_ylabel('Weight, $v_t$',loc='top', rotation = 0, labelpad = -82) 
#fig.savefig("GJRGARCH_weight_TC_best.png", bbox_inches = 'tight', pad_inches = 0.1)